In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import preprocess as ppx

Using TensorFlow backend.


In [2]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names)
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

In [3]:
imgs = np.array(train_images)

In [4]:
# adatok megfelelő formátumra hozása a keras számára
labels = np.array([ls[1] for ls in train_data])

encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[ 3  7  4 13  9  8  2  1 12  5  6  0 11 10]
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [5]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [6]:
# epoch szám
nb_epoch=100

In [7]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót
x = base_model.output
x = GlobalAveragePooling2D()(x)
# ezután hozzáadunk egy előrecsatolt réteget ReLU aktivációs függvénnyel
x = Dense(1024, activation='relu')(x)
# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(x)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf


In [8]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(imgs, labels_onehot, batch_size=8, nb_epoch=10, validation_split=0.2, callbacks=[history])

Train on 11 samples, validate on 3 samples
Epoch 1/10
11/11 [==============================] - 3s - loss: 2.7997 - acc: 0.0909 - val_loss: 4.7664 - val_acc: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 2s - loss: 0.3965 - acc: 0.9091 - val_loss: 4.9350 - val_acc: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 2s - loss: 0.0400 - acc: 1.0000 - val_loss: 5.1979 - val_acc: 0.0000e+00
Epoch 4/10
11/11 [==============================] - 2s - loss: 0.0351 - acc: 1.0000 - val_loss: 5.4493 - val_acc: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 2s - loss: 5.5623e-04 - acc: 1.0000 - val_loss: 5.6827 - val_acc: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 2s - loss: 1.5444e-04 - acc: 1.0000 - val_loss: 5.8434 - val_acc: 0.0000e+00
Epoch 7/10
11/11 [==============================] - 2s - loss: 0.1398 - acc: 0.9091 - val_loss: 6.1128 - val_acc: 0.0000e+00
Epoch 8/10
11/11 [==============================] - 2s - loss: 0.0515 - ac

In [10]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

In [11]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [12]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(np.asarray(train_images), labels_onehot,  batch_size=16, nb_epoch=10, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 11 samples, validate on 3 samples
Epoch 1/10
11/11 [==============================] - 4s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.2628 - val_acc: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.2844 - val_acc: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.3210 - val_acc: 0.0000e+00
Epoch 4/10
11/11 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.3666 - val_acc: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.4078 - val_acc: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.4345 - val_acc: 0.0000e+00
Epoch 7/10
11/11 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 7.4729 - val_acc: 0.0000e+00
Epoch 8/10
11/11 [==============================] - 2s

In [13]:
predicted_values = model.predict(np.asarray(train_images))

In [18]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1

In [19]:
encoder.inverse_transform(max_pred).shape

(14, 14)

In [22]:
# vissza kéne alakítani valahogy...
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])\
.reshape(n_samples, n_features) - ohc.feature_indices_[:-1]

NameError: name 'out' is not defined